In [18]:
import random
import itertools
import json
import os
random.seed(1)
with open('train.json', 'r') as file:
    train_dataset = json.load(file)
with open('test.json', 'r') as file:
    test_dataset = json.load(file)

In [ ]:
def reorder_equations(data,order='forward',split='train'):
    eqs = data['premise'].split("\n")
    reorder_eqs =[]
    if order=='forward':
        reorder_eqs.append("\n".join(eqs+[f"{data['target']}>>"]))
    elif order=='reverse':
        reorder_eqs.append("\n".join(eqs[::-1]+[f"{data['target']}>>"]))
    elif order=='random':
        eqs_list=[perm for perm in itertools.permutations(eqs)]
        random.shuffle(eqs_list)
        if split=='train':
            for eq in eqs_list[:5]:
                reorder_eqs.append("\n".join(list(eq)+[f"{data['target']}>>"]))
        else:
            for eq in eqs_list[:1]:
                reorder_eqs.append("\n".join(list(eq)+[f"{data['target']}>>"]))
    else:
        print('Please input correct order.')
        raise(NotImplementedError)
    return reorder_eqs

train_dataset[0],reorder_equations(train_dataset[0],'random','train')

({'template': 'num0_=19-5\nnum1_=num0_+15\nnum2_=16+num1_',
  'premise': 'y=19-5\no=y+15\nj=16+o',
  'target': 'j',
  'gt': 22,
  'step': '3',
  'type': 'train'},
 ['o=y+15\ny=19-5\nj=16+o\nj>>',
  'o=y+15\nj=16+o\ny=19-5\nj>>',
  'j=16+o\no=y+15\ny=19-5\nj>>',
  'y=19-5\no=y+15\nj=16+o\nj>>',
  'j=16+o\ny=19-5\no=y+15\nj>>'])

In [24]:
order='random'
save_dir=f'data_{order}'

if not os.path.exists(f'../LLaMA-Factory-main/data/{save_dir}'):  
    os.makedirs(f'../LLaMA-Factory-main/data/{save_dir}') 

ouf=[]
for item in train_dataset:
    for e in reorder_equations(item,order=order,split='train'):
        ouf.append({'instruction': e,
                    'input': '',
                    'output': item['gt']})
print(len(ouf))
random.shuffle(ouf)
with open(f'../LLaMA-Factory-main/data/{save_dir}/train.json', 'w') as file:
    json.dump(ouf, file, indent=4)

ouf=[]
for item in test_dataset:
    for e in reorder_equations(item,order=order,split='test'):
        ouf.append({'instruction': e,
                    'input': '',
                    'output': item['gt']})
print(len(ouf))
with open(f'../LLaMA-Factory-main/data/{save_dir}/test.json', 'w') as file:
    json.dump(ouf, file, indent=4)

with open('../LLaMA-Factory-main/data/dataset_info.json') as f:
    dataset_info=json.load(f)
if save_dir+'-train' in dataset_info or save_dir+'-test' in dataset_info:
    print('data_info exists!')
else:
    dataset_info[save_dir+'-train']={'file_name':f'{save_dir}/train.json'}
    dataset_info[save_dir+'-test']={'file_name':f'{save_dir}/test.json'}

    with open('../LLaMA-Factory-main/data/dataset_info.json','w') as f:
        json.dump(dataset_info, f, indent=2, ensure_ascii=False)

850428
3800
